<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Données" data-toc-modified-id="Données-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Données</a></span></li></ul></li><li><span><a href="#Import" data-toc-modified-id="Import-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import</a></span><ul class="toc-item"><li><span><a href="#Outils" data-toc-modified-id="Outils-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Outils</a></span></li></ul></li><li><span><a href="#S3-aws-avec-boto3" data-toc-modified-id="S3-aws-avec-boto3-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>S3 aws avec boto3</a></span><ul class="toc-item"><li><span><a href="#Credentials" data-toc-modified-id="Credentials-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Credentials</a></span></li></ul></li><li><span><a href="#Session-Spark" data-toc-modified-id="Session-Spark-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Session Spark</a></span><ul class="toc-item"><li><span><a href="#Info-sur-spark-et-la-session" data-toc-modified-id="Info-sur-spark-et-la-session-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Info sur spark et la session</a></span></li></ul></li><li><span><a href="#Load-dataframe" data-toc-modified-id="Load-dataframe-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load dataframe</a></span><ul class="toc-item"><li><span><a href="#Dataframe-des-images-dans-S3" data-toc-modified-id="Dataframe-des-images-dans-S3-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Dataframe des images dans S3</a></span></li><li><span><a href="#Dataframe-des-images-en-locale" data-toc-modified-id="Dataframe-des-images-en-locale-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Dataframe des images en locale</a></span></li></ul></li><li><span><a href="#Resize" data-toc-modified-id="Resize-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Resize</a></span><ul class="toc-item"><li><span><a href="#Label-de-l'image" data-toc-modified-id="Label-de-l'image-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Label de l'image</a></span></li></ul></li><li><span><a href="#Reduction-avec-PCA" data-toc-modified-id="Reduction-avec-PCA-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Reduction avec PCA</a></span><ul class="toc-item"><li><span><a href="#Création-d'un-vecteur-dense" data-toc-modified-id="Création-d'un-vecteur-dense-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Création d'un vecteur dense</a></span></li><li><span><a href="#MinMaxScaler-Scaler" data-toc-modified-id="MinMaxScaler-Scaler-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>MinMaxScaler Scaler</a></span></li><li><span><a href="#PCA" data-toc-modified-id="PCA-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>PCA</a></span></li><li><span><a href="#Save-Dataframe" data-toc-modified-id="Save-Dataframe-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Save Dataframe</a></span></li></ul></li><li><span><a href="#GIT" data-toc-modified-id="GIT-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>GIT</a></span></li><li><span><a href="#Bibliography" data-toc-modified-id="Bibliography-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Bibliography</a></span></li></ul></div>

# Introduction

Dans un environnement Big Data, une première chaîne de traitement des données qui comprendra le preprocessing et une étape de réduction de dimension.
La sortie de la réduction de dimension (une matrice écrite sur un fichier CSV ou autre) est ecrite dans un espace de stockage sur le cloud.



## Données

https://www.kaggle.com/datasets/moltean/fruits?select=fruits-360_dataset


# Import


In [1]:
# pyspark
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import split, udf, col, pandas_udf, PandasUDFType
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, BinaryType, DoubleType, DataType, FloatType
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import PCA, MinMaxScaler
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector

from typing import Iterator

# image
from PIL import Image, ImageDraw

# Outils
import numpy as np
import pandas as pd
import sys
import os
import io
import time
from contextlib import contextmanager


import boto3


## Outils


In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))


In [3]:
s3bucket = 'ocunderscorep8'
debug_folder = 'fruits-360-debug/Training/'
full_folder = 'fruits-360/Training/'

# S3 aws avec boto3


https://us-east-1.console.aws.amazon.com/iam/home#/users$new?step=permissions&accessKey&userNames=app_p8&permissionType=policies

In [4]:
session = boto3.session.Session(aws_access_key_id="",
                                        aws_secret_access_key="")
s3_client = session.client(service_name='s3')
prefix = debug_folder #fruits-360/Training/
sub_folders = s3_client.list_objects_v2(Bucket=s3bucket, Prefix=prefix)


Mes buckets sur s3

In [5]:
s3_client.list_buckets()

{'ResponseMetadata': {'RequestId': '5831TKXGF80ZJWX4',
  'HostId': 'M6aTYDm+TNf+wslJb2zpSq/Xi3Xhz2AV09EC0c5tos4qzQ4ltBvyGqJFT2YDJQ3S4Q4AgTHWe+Y=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'M6aTYDm+TNf+wslJb2zpSq/Xi3Xhz2AV09EC0c5tos4qzQ4ltBvyGqJFT2YDJQ3S4Q4AgTHWe+Y=',
   'x-amz-request-id': '5831TKXGF80ZJWX4',
   'date': 'Fri, 25 Nov 2022 18:11:59 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aws-logs-479126531994-eu-west-3',
   'CreationDate': datetime.datetime(2022, 11, 13, 21, 40, 19, tzinfo=tzutc())},
  {'Name': 'ocunderscorep8',
   'CreationDate': datetime.datetime(2022, 11, 11, 15, 9, 24, tzinfo=tzutc())},
  {'Name': 'sagemaker-studio-479126531994-bb9tbgaoy2h',
   'CreationDate': datetime.datetime(2022, 11, 19, 21, 26, 35, tzinfo=tzutc())}],
 'Owner': {'DisplayName': 'isvillni',
  'ID': '6f787a108b861a0c711fcad4e301e7d39d745f76275b17fc6f38c82563b80cb0'}}

Mes subfolder sur S3

In [6]:
sub_folders

{'ResponseMetadata': {'RequestId': '8DW2WD8GCBN49N2G',
  'HostId': 'jbbOUetmweWowVK6JbTjBd9jWVxdMdYK+EZLeTzM2b9JOo14UVPh4cyQpBDU8z+A4VYDush3LPN/TH/ry7GL7Q==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'jbbOUetmweWowVK6JbTjBd9jWVxdMdYK+EZLeTzM2b9JOo14UVPh4cyQpBDU8z+A4VYDush3LPN/TH/ry7GL7Q==',
   'x-amz-request-id': '8DW2WD8GCBN49N2G',
   'date': 'Wed, 23 Nov 2022 18:39:32 GMT',
   'x-amz-bucket-region': 'eu-west-3',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'IsTruncated': True,
 'Contents': [{'Key': 'fruits-360-debug/Training/Apple_Braeburn/0_100.jpg',
   'LastModified': datetime.datetime(2022, 11, 23, 16, 2, 8, tzinfo=tzutc()),
   'ETag': '"2247c5e506d29555ba9bcd403cf29330"',
   'Size': 4773,
   'StorageClass': 'STANDARD'},
  {'Key': 'fruits-360-debug/Training/Apple_Braeburn/100_100.jpg',
   'LastModified': datetime.datetime(2022, 11, 23, 16, 2, 8, tzinfo=tzutc()),
   'ETag': '"324fa5ff98952

In [7]:
paths =  []
for key in sub_folders["Contents"]:
    file = key["Key"]
    paths.append("s3://ocunderscorep8/" + file)
paths

['s3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/0_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/100_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/101_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/102_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/103_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/104_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/105_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/106_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/107_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/108_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/109_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/10_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple_Braeburn/110_100.jpg',
 '

## Credentials

J'utilise mes credentials root, ce qui en soi est "mal". Mais il s'agit d'un petit projet sans suite.

In [5]:
credentials = session.get_credentials()
AWS_ACCESS_KEY_ID = credentials.access_key
AWS_SECRET_ACCESS_KEY = credentials.secret_key

# Session Spark



In [6]:
spark = SparkSession.builder.appName("oc_p8").config("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID).config("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY).config("spark.driver.memory", "15g").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

22/11/26 16:05:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Info sur spark et la session

In [7]:

spark


In [8]:
sc = spark.sparkContext
print(f"Hadoop version = {sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")

Hadoop version = 3.2.0


# Load dataframe

In [9]:
def load_data(path):
    #chargement dataframe des images
    df_img = spark.read.format("image").option("recursiveFileLookup", True).option("dropInvalid", True).load(path)
    print(df_img.printSchema())
    return df_img

## Dataframe des images dans S3

J'ai du ajouter 3 jars qui manquanient. Voir bibliographie.



In [11]:
df2_img =load_data('s3a://ocunderscorep8/fruits-360-debug/Training/*')

22/11/26 00:10:01 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

None


In [12]:
df2_img.select("image.origin", "image.height", "image.width", "image.nChannels", "image.mode", "image.data" ).show(truncate=30)

+------------------------------+------+-----+---------+----+------------------------------+
|                        origin|height|width|nChannels|mode|                          data|
+------------------------------+------+-----+---------+----+------------------------------+
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FD FF FF FD FF FF FD FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF F

## Dataframe des images en locale



In [10]:


df =load_data(debug_folder)


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

None


In [11]:
df.select("image.origin", "image.height", "image.width", "image.nChannels", "image.mode", "image.data" ).show(truncate=30)

+------------------------------+------+-----+---------+----+------------------------------+
|                        origin|height|width|nChannels|mode|                          data|
+------------------------------+------+-----+---------+----+------------------------------+
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FD FF FF FD FF FF FD FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF F

Les données de l'image sont un array de 30000 elements (100\*100\*3)

In [12]:
row=df.first()
print(len(row[0].__getitem__('data')))

30000


Affichage de l'image de la première ligne, une pastèque

In [13]:
spark_single_img = df.first()
(spark_single_img.image.origin, spark_single_img.image.mode, spark_single_img.image.nChannels )

mode = 'RGBA' if (spark_single_img.image.nChannels == 4) else 'RGB' 
Image.frombytes(mode=mode, data=bytes(spark_single_img.image.data), size=[spark_single_img.image.width,spark_single_img.image.height]).show()



de BGR à RGB : une pastèque moins bleutée

In [14]:
def convert_bgr_array_to_rgb_array(img_array):
    B, G, R = img_array.T
    return np.array((R, G, B)).T

img = Image.frombytes(mode=mode, data=bytes(spark_single_img.image.data), size=[spark_single_img.image.width,spark_single_img.image.height])

converted_img_array = convert_bgr_array_to_rgb_array(np.asarray(img))
Image.fromarray(converted_img_array).show()

# Resize

In [15]:
schema = StructType(df.select("image.*").schema.fields + [
    StructField("data_as_resized_array", ArrayType(IntegerType()), True),
    StructField("data_as_array", ArrayType(IntegerType()), True)
])

def resize_img(img_data, resize=True):
    mode = 'RGBA' if (img_data.nChannels == 4) else 'RGB' 
    img = Image.frombytes(mode=mode, data=img_data.data, size=[img_data.width, img_data.height])
    img = img.convert('RGB') if (mode == 'RGBA') else img
    img = img.resize([40, 40], resample=Image.Resampling.BICUBIC) if (resize) else img
    arr = convert_bgr_array_to_rgb_array(np.asarray(img))
    arr = arr.reshape([40*40*3]) if (resize) else arr.reshape([img_data.width*img_data.height*3])

    return arr

def resize_image_udf(dataframe_batch_iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for dataframe_batch in dataframe_batch_iterator:
        dataframe_batch["data_as_resized_array"] = dataframe_batch.apply(resize_img, args=(True,), axis=1)
        dataframe_batch["data_as_array"] = dataframe_batch.apply(resize_img, args=(False,), axis=1)
        yield dataframe_batch

resized_df = df.select("image.*").mapInPandas(resize_image_udf, schema)

In [16]:
row=resized_df.first()
print(len(row.__getitem__('data_as_resized_array')))

4800


In [17]:
df=resized_df

In [18]:
resized_df.unpersist()

DataFrame[origin: string, height: int, width: int, nChannels: int, mode: int, data: binary, data_as_resized_array: array<int>, data_as_array: array<int>]

## Label de l'image

Je simplifie les labels avec un udf. Toutes les sous-catégories (ex. Pear 1, Pear 2)sont groupées dans une catégorie (ex. Pear).



In [19]:
def set_label(path):
        return path.split('/')[-2].split('_')[0]
udf_label = udf(set_label, StringType())

def labelize(df):
    return df.withColumn('Label', udf_label("origin"))

df = labelize(df)



In [20]:
df.show()

+--------------------+------+-----+---------+----+--------------------+---------------------+--------------------+----------+
|              origin|height|width|nChannels|mode|                data|data_as_resized_array|       data_as_array|     Label|
+--------------------+------+-----+---------+----+--------------------+---------------------+--------------------+----------+
|file:///Users/mcB...|   100|  100|        3|  16|[FF FF FF FF FF F...| [255, 255, 255, 2...|[255, 255, 255, 2...|Watermelon|
|file:///Users/mcB...|   100|  100|        3|  16|[FD FF FF FD FF F...| [255, 255, 253, 2...|[255, 255, 253, 2...|Watermelon|
|file:///Users/mcB...|   100|  100|        3|  16|[FF FF FF FF FF F...| [255, 255, 255, 2...|[255, 255, 255, 2...|Watermelon|
|file:///Users/mcB...|   100|  100|        3|  16|[FF FF FF FF FF F...| [255, 255, 255, 2...|[255, 255, 255, 2...|Watermelon|
|file:///Users/mcB...|   100|  100|        3|  16|[FF FF FF FF FF F...| [255, 255, 255, 2...|[255, 255, 255, 2...|Wate

# Reduction avec PCA

Inspiré par : https://stackoverflow.com/questions/73654149/pyspark-image-dimension-reduction-with-pca

## Création d'un vecteur dense

à partir de image.data. image.data est un vecteur : https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/ml/image/ImageSchema.scala#L131-L190 With a 640 x 480 RGB image example, the first three positions of the array will contain color information(Blue, Green and Red) from the pixel at height = 0 and width = 0. The next three position will contain information from the pixel at at height = 0 and width = 1 and so on. (https://towardsdatascience.com/image-segmentation-on-apache-spark-46164dd53c73)

In [21]:
ud_img2vec = udf(lambda x : Vectors.dense(x), VectorUDT())

df = df.withColumn("data_as_vector", ud_img2vec("data_as_resized_array"))



In [22]:
df.show()

+--------------------+------+-----+---------+----+--------------------+---------------------+--------------------+----------+--------------------+
|              origin|height|width|nChannels|mode|                data|data_as_resized_array|       data_as_array|     Label|      data_as_vector|
+--------------------+------+-----+---------+----+--------------------+---------------------+--------------------+----------+--------------------+
|file:///Users/mcB...|   100|  100|        3|  16|[FF FF FF FF FF F...| [255, 255, 255, 2...|[255, 255, 255, 2...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|   100|  100|        3|  16|[FD FF FF FD FF F...| [255, 255, 253, 2...|[255, 255, 253, 2...|Watermelon|[255.0,255.0,253....|
|file:///Users/mcB...|   100|  100|        3|  16|[FF FF FF FF FF F...| [255, 255, 255, 2...|[255, 255, 255, 2...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|   100|  100|        3|  16|[FF FF FF FF FF F...| [255, 255, 255, 2...|[255, 255, 255, 2...|Wate

In [23]:
df=df.select('origin', 'Label', 'data_as_vector')
df.show()

+--------------------+----------+--------------------+
|              origin|     Label|      data_as_vector|
+--------------------+----------+--------------------+
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,253....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[254.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///U

## MinMaxScaler Scaler


**/!\ Il faut modifer les noms de folder et remplacer les espaces par des underscore, sinon on a une erreur dans le fit.**

In [24]:
#standardizer = StandardScaler(withMean=True, withStd=True, inputCol="data_as_vector", outputCol="scaledFeatures")
standardizer = MinMaxScaler(inputCol="data_as_vector", 
                            outputCol="scaledFeatures",
                            min=-1.0,
                            max=1.0)
model_std = standardizer.fit(df)
df = model_std.transform(df)


In [25]:
df.show()

+--------------------+----------+--------------------+--------------------+
|              origin|     Label|      data_as_vector|      scaledFeatures|
+--------------------+----------+--------------------+--------------------+
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,253....|[1.0,1.0,0.428571...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|
|file:///Use


## PCA

22/11/14 17:18:18 WARN RowMatrix: 30000 columns will require at least 7200 megabytes of memory!

In [26]:
# algorithm
pca = PCA(k=2, inputCol='scaledFeatures', outputCol='pcaFeatures')
model_pca = pca.fit(df)



22/11/26 16:09:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/26 16:09:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/11/26 16:11:47 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/11/26 16:11:47 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [29]:
# Transformation images
df = model_pca.transform(df)
    
df.show()

+--------------------+----------+--------------------+--------------------+--------------------+
|              origin|     Label|      data_as_vector|      scaledFeatures|         pcaFeatures|
+--------------------+----------+--------------------+--------------------+--------------------+
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|[8.98615364755610...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,253....|[1.0,1.0,0.428571...|[4.81707362510396...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|[7.77561801779438...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|[8.82803862775854...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|[8.45615177186154...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|[8.51858110964552...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[1.0,1.0,1.0,1.0,...|[8.09582382716425...|
|file:///Users/mcB...|Watermel

## Save Dataframe


In [38]:
def write_where(path='local'):
    if path == 'S3':
        return 's3a://ocunderscorep8/fruits-360-debug/parquet.parquet'
    else :
        return './parquet.parquet'

path=write_where()
df.write.format("parquet").mode('overwrite').save(path)


# GIT

https://github.com/cbedini/oc_p8


# Bibliography


Spark

https://medium.com/@AaronSchlegel/install-spark-on-macos-and-load-data-from-aws-s3-with-pyspark-6884bd5f0d90


Pyspark

https://livebook.manning.com/book/pyspark-in-action/chapter-3/1

https://anujsyal.com/pyspark-installation-guide


Example

https://docs.databricks.com/_static/notebooks/deep-learning/deep-learning-transfer-learning-keras.html


Spark Context, Session

https://medium.com/@achilleus/spark-session-10d0d66d1d24


AWS CLI envvars

https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-envvars.html


CLI, Boto3

https://www.carolynlangen.com/blog/2017-11-22-interacting-with-aws-s3-using-python-in-a-jupyter-notebook/


Boto3

https://medium.com/@antonysruthy11/loading-kaggle-dataset-to-aws-s3-using-boto3-50af3e015fb2

https://towardsdatascience.com/reading-and-writing-files-from-to-amazon-s3-with-pandas-ccaf90bfe86c




Images processing

https://godatadriven.com/blog/real-distributed-image-processing-with-apache-spark/

https://stackoverflow.com/questions/69205589/how-preprocess-image-using-pyspark

https://www.databricks.com/blog/2018/12/10/introducing-built-in-image-data-source-in-apache-spark-2-4.html

https://spark.apache.org/docs/latest/ml-datasource.html


UDF

https://stackoverflow.com/questions/50222362/pyspark-split-the-string-column-and-join-part-of-them-to-form-new-columns


PCA

https://people.duke.edu/~ccc14/sta-663-2016/21D_Spark_MLib.html

https://stackoverflow.com/questions/73654149/pyspark-image-dimension-reduction-with-pca


Upload data sur S3

https://manan.au/data-platforms/amazon-aws/loading-data-from-kaggle-to-amazon-s3-bucket/


s3a

https://stackoverflow.com/questions/58415928/spark-s3-error-java-lang-classnotfoundexception-class-org-apache-hadoop-f

https://stackoverflow.com/questions/64563127/pyspark-s3-error-java-lang-noclassdeffounderror-com-amazonaws-services-s3-mode

http://deploymentzone.com/2015/12/20/s3a-on-spark-on-aws-ec2/

https://medium.com/@erkansirin/apache-spark-aws-s3-datasource-eb65ee906e64

https://medium.com/@AaronSchlegel/install-spark-on-macos-and-load-data-from-aws-s3-with-pyspark-6884bd5f0d90

https://notadatascientist.com/running-apache-spark-and-s3-locally/


Sagemaker et SparkMagic

https://www.google.com/search?q=run+pyspark+noebook+on+sagemaker+studio&oq=run+pyspark+noebook+on+sagemaker&aqs=chrome.1.69i57j33i10i160j33i22i29i30.14697j0j7&sourceid=chrome&ie=UTF-8#fpstate=ive&vld=cid:6652f735,vid:RxRENYQBxZU


Alternative à SageMaker

https://towardsdatascience.com/setting-up-and-using-jupyter-notebooks-on-aws-61a9648db6c5

https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-managed-notebooks-headless-cli.html

https://medium.com/swlh/running-pyspark-applications-on-amazon-emr-e536b7a865ca

SSH

https://modelpredict.com/sagemaker-ssh-setup/

PCA

https://medium.com/@david.kabii/pyspark-feature-engineering-and-high-dimensional-data-visualization-with-spark-sql-in-an-hour-b2fea0de2472


Alternative à SageMaker

https://towardsdatascience.com/setting-up-and-using-jupyter-notebooks-on-aws-61a9648db6c5

https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-managed-notebooks-headless-cli.html

https://medium.com/swlh/running-pyspark-applications-on-amazon-emr-e536b7a865ca
